## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
project = pd.read_csv('/datasets/data.csv')
#project.head(15)
project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод: в файле всего 21525 строк, самый проблемный столбец days_employed, так как в нем стоят отрицательные значения и для пенсионеров, кол-во дней трудового стажа исчисляются в 1000 лет и имеются пропуски. Данный столбец не потребуется для реализации нашего проекта и его можно удалить, но нужно предупредить коллег, чтобы в будущем не сталкиваться с подобной проблемой. В остальном есть пропуски в столбце total_income. И проблемы со шрифтом в столбце education.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#project['total_income'].mean()
project['total_income'] = project['total_income'].fillna(project['total_income'].mean())
#print(project['dob_years'][project['dob_years'] == 0].count())
#project.info()
#project['dob_years'].mean()
project['dob_years'] = project['dob_years'].replace(0, 43.29337979094077)
#project['dob_years'][project['dob_years'] == 0].count()
#project['dob_years'].min()
project.dropna(axis = 'columns', inplace = True)
def child_final(i):
    if i == 0:
        return 0
    if i == 20:
        return 2
    if i == 1:
        return 1
    if i == 3:
        return 3
    if i == 4:
        return 4
    if i == 5:
        return 5
    else:
        return 2

    
project['children_final'] = project['children'].apply(child_final)
project.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
children_final      21525 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод: Заполнил пропуски в столбце total_income на среднее значение по столбцу и заменил нулевой возраст в столбце dob_years на средний возраст. Удалили столбец days_employed, так как он не требуется для анализа, заметил, что этот столбец и столбец total_income имеют одинаковое количество строк, значит они взаимозависимы, но большее сказать не могу, ничего в голову не приходит. Возможно, что пропуски появились из-за того, что клиенты банка не указывали свой стаж и месячный доход. Либо не указывался стаж и соотвественно не подтягивались данные по месячному доходу. Ввел функцию для исправления значения в столбце children, где было 20 детей. Короче кода не придумал, подскажите, знаю, что можно короче не не знаю как, много вариантов испробовал.

<font color='green'>Гипотезы правдоподобны, хорошо, что ты их выдвинул.</font>

### Замена типа данных

In [3]:
project['dob_years'] = project['dob_years'].astype('int')
project['total_income'] = project['total_income'].astype('int')
project.head(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_final
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,2
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0


### Вывод: Были скорректированы столбцы total_income  и dob_years в формат int. Для того, чтобы таблица легче воспринималась на глаз.

<font color='green'>С типами данных все хорошо.</font>

### Обработка дубликатов

In [4]:
project['education'] = project['education'].str.lower()
#project.head(15)
#print(project.duplicated().sum())
#project['education'].value_counts()
project = project.drop_duplicates()
project.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
children_final      21454 non-null int64
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод:  В столбце education преобразованны значения в строках на прописные буквы, после чего было найдено 71 дубликат. Все дубликаты имеют одно и то же значение по столбцу total_income. По поводу гипотезы совсем никаких идей.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()

def lemmatizator(i):
    lemmas = m.lemmatize(i)                     
    return lemmas

project['purpose_lemmas'] = project['purpose'].apply(lemmatizator)
project.tail(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_final,purpose_lemmas
21510,2,28,среднее,1,женат / замужем,0,F,сотрудник,0,167422,приобретение автомобиля,2,"[приобретение, , автомобиль, \n]"
21511,0,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068,покупка жилья для сдачи,0,"[покупка, , жилье, , для, , сдача, \n]"
21512,0,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301,получение дополнительного образования,0,"[получение, , дополнительный, , образование,..."
21513,0,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986,покупка жилья,0,"[покупка, , жилье, \n]"
21514,0,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988,строительство недвижимости,0,"[строительство, , недвижимость, \n]"
21515,1,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,1,"[заниматься, , образование, \n]"
21516,0,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,0,"[покупка, , свой, , жилье, \n]"
21517,0,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,0,"[на, , покупка, , свой, , автомобиль, \n]"
21518,0,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,0,"[сделка, , с, , автомобиль, \n]"
21519,1,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,1,"[покупка, , коммерческий, , недвижимость, \n]"


### Вывод: Леммитизация производилась в отдельном столбце, для того, чтобы была возможность категоризировать цели.

<font color='green'>Лемматизация проведена правильно, молодец.</font>

### Категоризация данных

In [6]:
def replace(i):
    if 'свадьба' in i:
        return 'свадьба'
    elif 'автомобиль' in i:
        return 'авто'
    elif 'недвижимость' in i or 'жилье' in i:
        return 'недвижимость'
    elif 'образование' in i:
        return 'образование'
    else:
        return 5
    
project['purpose_id'] = project['purpose_lemmas'].apply(replace)
project['purpose_id'].value_counts()


def replace_income(total_income):
    if total_income >= 20667 and total_income <= 64145:
        return 'низкий'
    if total_income > 64145 and total_income <= 107623:
        return 'средний'
    if total_income > 107623 and total_income <= 156044:
        return 'выше среднего'
    if total_income > 156044 and total_income <= 195813:
        return 'высокий'
    if total_income > 195813:
        return 'очень высокий'
    
    
project['total_income_id'] = project['total_income'].apply(replace_income)


def age(dob_years):
    if dob_years <=30:
        return 'молодые'
    if dob_years >=31 and dob_years <=55:
        return 'взрослые'
    if dob_years >=56:
        return 'взрослые'
project['age'] = project['dob_years'].apply(age)
project

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_final,purpose_lemmas,purpose_id,total_income_id,age
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,"[покупка, , жилье, \n]",недвижимость,очень высокий,взрослые
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,"[приобретение, , автомобиль, \n]",авто,выше среднего,взрослые
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,"[покупка, , жилье, \n]",недвижимость,выше среднего,взрослые
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,"[дополнительный, , образование, \n]",образование,очень высокий,взрослые
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0,"[сыграть, , свадьба, \n]",свадьба,высокий,взрослые
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,1,"[операция, , с, , жилье, \n]",недвижимость,очень высокий,взрослые
21521,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,0,"[сделка, , с, , автомобиль, \n]",авто,выше среднего,взрослые
21522,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,1,"[недвижимость, \n]",недвижимость,средний,взрослые
21523,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,3,"[на, , покупка, , свой, , автомобиль, \n]",авто,очень высокий,взрослые


### Вывод: Категоризацию проводил с соотвествием лемм по столбцу puprpose_lemmas. Выбрал 4 основные категории. Что касается категоризации по столбцу total_income, взял статью, где описанны доходы граждан и применил к своему коду, решил взять текстовый вариант, ты прав, так намного легче анализировать данные! Решил добавить столбец age, для проверки зависимости семейного положение на выплату по кредиту в срок, так как возраст дает логическую привязку к жизненным обстоятельствам и ответсвенности клеинтов, а так же для остального анализа, так как возраст имеет зависимость в каждом анализе. Эта категория будет использованна в основном выводе, как доп.аргумент. Использовал метод describe().round() для поиска квартилей, категорию низкий получил с помощью того, что нашел среднее значение между минимальным значением и 25%. 

<font color=green> Хорошо, все доработал как надо</font>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#project['children'].value_counts()
#заметил, что есть отрицательные значения и вызывает сомнение цифра 20, но все возможно))
project['children'] = project['children'].abs()
#project['children'].value_counts()
#отрицательные значения измененя на положительные
#mean = project.groupby('children')['debt'].mean()
#def children_count(series):
    #return "(0:.2%)".format(series.sum()/series.count())
#project.groupby('children').agg({'children':['mean', 'count', children_count]})
#project.groupby('children')['debt'].mean()
ch = project.pivot_table(index = ['children_final'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

def percent(x):
    return "{0:.2%}".format(x)

ch['mean'] = ch['mean'].apply(percent)
ch = ch.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
ch.sort_values(by='Количество записей', ascending=False)


,Количество записей,Среднее
children_final,,
0,14091.0,7.54%
1,4808.0,9.23%
2,2175.0,9.33%
3,330.0,8.18%
4,41.0,9.76%
5,9.0,0.00%


### Вывод: Из данной таблицы можно сделать вывод, что клиенты без детей отвественнее всего подходят к выплате кредита в срок, впринципе, в этом есть логика, так как у них меньше ежемесячных затрат. На втором месте расположились клиенты с 3-я детьми, мои предположения в том, что семьи с 3-я детьми более отвественны, чем семьи с 1-2 детьми, так как им наверняка потребуется хорошая кредитная история для будущих кредитов, например для детей. На 3 ,4 и 5 местах расположились семьи с 1-м, 2-я и 4-я детьм соотвественно, опираясь на логику описанную в предидущем предложении, могу предположить, что семьи с 1-м и 2-я детьми берут кредит одноразово, поэтому не так сильно ориентируются на срок выплат. Разница между минимальным и максимальным средним значением составляет 23,1%.

<font color=green> Теперь все верно, молодец. Красивый вывод процентов. </font>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
#print(project.groupby('family_status')['debt'].mean())
fs = project.pivot_table(index = ['family_status'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

def percent(x):
    return "{0:.2%}".format(x)

fs['mean'] = fs['mean'].apply(percent)
fs = fs.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
fs.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее
family_status,,
женат / замужем,12339.0,7.55%
гражданский брак,4151.0,9.35%
Не женат / не замужем,2810.0,9.75%
в разводе,1195.0,7.11%
вдовец / вдова,959.0,6.57%


### Вывод: Как видно из таблицы, вдовец/вдова самые отвественные из клиентов. Разница между минимальным значением 6.56% и максимальным 9.74% составляет 32,6% процентов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
inc = project.pivot_table(index = 'total_income_id', values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))
def percent(x):
    return "{0:.2%}".format(x)

inc['mean'] = inc['mean'].apply(percent)
inc = inc.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
inc.sort_values(by='Количество записей', ascending=False)


,Количество записей,Среднее по задолжностям
total_income_id,,
очень высокий,5364.0,7.14%
высокий,5363.0,8.63%
выше среднего,5363.0,8.73%
средний,4323.0,8.28%
низкий,1041.0,6.63%


### Вывод: Лушчий средний показатель имеют клиенты с низким доходом. Рассуждая логически, клиенты с низким доходом понимают, что в им нужна положительная кредитная история, для того, чтобы одобрили следующий. Разница в данной категории между минимальным и максимальным значениями равна 28,2%. 

<font color=green> Вывод из таких данных верный, но стоит обращать внимание, на количество записей в катеогриях. Низкого дохода у нас довольно мало, возможно, их бы стоило объединить со средней группой. Опять-такги, здесь бы помогли квантили. </font>

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
#ps = project.pivot_table(index = ['purpose_id', 'age'], values='debt', aggfunc = ('mean', lambda X:
                                                                       #X.count()))
ps = project.pivot_table(index = ['purpose_id'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))
def percent(x):
    return "{0:.2%}".format(x)

ps['mean'] = ps['mean'].apply(percent)
ps = ps.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
ps.sort_values(by='Количество записей', ascending=False)


,Количество записей,Среднее по задолжностям
purpose_id,,
недвижимость,10811.0,7.23%
авто,4306.0,9.36%
образование,4013.0,9.22%
свадьба,2324.0,8.00%


### Вывод: Луший показатель у клиентов с целью покупки недвижимости. Разница между минимальным и максимальными средними значениями равна 22,8%.


### Шаг 4. Общий вывод
Все категории имеют зависимость с выплатой кредита в срок.
1. Семейное положение - 32,6%
2. Уровень дохода - 28,2%
3. Количество детей в семье - 23.1%
4. Цели на кредит - 22.8%
То есть Категория - Большее внимание нужно обращать на категрию Семейное положение.

In [11]:
ch = project.pivot_table(index = ['children_final', 'age'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

def percent(x):
    return "{0:.2%}".format(x)

ch['mean'] = ch['mean'].apply(percent)
ch = ch.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
ch.sort_values(by='Количество записей', ascending=False)

Количество записей Среднее
children_final age                                 
0              взрослые             11944.0   6.94%
1              взрослые              3753.0   8.79%
0              молодые               2147.0  10.90%
2              взрослые              1716.0   8.86%
1              молодые               1055.0  10.81%
2              молодые                459.0  11.11%
3              взрослые               280.0   8.57%
               молодые                 50.0   6.00%
4              взрослые                35.0   8.57%
5              взрослые                 9.0   0.00%
4              молодые                  6.0  16.67%

In [12]:
fs = project.pivot_table(index = ['family_status', 'age'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

def percent(x):
    return "{0:.2%}".format(x)

fs['mean'] = fs['mean'].apply(percent)
fs = fs.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
fs.sort_values(by='Количество записей', ascending=False)

,,Количество записей,Среднее
family_status,age,,
женат / замужем,взрослые,10545.0,7.06%
гражданский брак,взрослые,3366.0,9.03%
Не женат / не замужем,взрослые,1796.0,8.91%
женат / замужем,молодые,1794.0,10.42%
в разводе,взрослые,1078.0,6.31%
Не женат / не замужем,молодые,1014.0,11.24%
вдовец / вдова,взрослые,952.0,6.51%
гражданский брак,молодые,785.0,10.70%
в разводе,молодые,117.0,14.53%


In [13]:
inc = project.pivot_table(index = ['total_income_id', 'age'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))
def percent(x):
    return "{0:.2%}".format(x)

inc['mean'] = inc['mean'].apply(percent)
inc = inc.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
inc.sort_values(by='Количество записей', ascending=False)

,,Количество записей,Среднее по задолжностям
total_income_id,age,,
очень высокий,взрослые,4528.0,6.87%
высокий,взрослые,4408.0,8.12%
выше среднего,взрослые,4374.0,7.96%
средний,взрослые,3526.0,7.60%
выше среднего,молодые,989.0,12.13%
высокий,молодые,955.0,10.99%
низкий,взрослые,901.0,5.88%
очень высокий,молодые,836.0,8.61%
средний,молодые,797.0,11.29%


In [14]:
ps = project.pivot_table(index = ['purpose_id', 'age'], values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))
def percent(x):
    return "{0:.2%}".format(x)

ps['mean'] = ps['mean'].apply(percent)
ps = ps.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
ps.sort_values(by='Количество записей', ascending=False)



,,Количество записей,Среднее по задолжностям
purpose_id,age,,
недвижимость,взрослые,8891.0,6.68%
авто,взрослые,3620.0,8.81%
образование,взрослые,3294.0,8.41%
свадьба,взрослые,1932.0,7.66%
недвижимость,молодые,1920.0,9.79%
образование,молодые,719.0,12.93%
авто,молодые,686.0,12.24%
свадьба,молодые,392.0,9.69%


### Дополнительный вывод: Так же для построение более точной скорринговой модели нужно использовать в сводной таблице столбец с возрастом клиентов, он меняет картину и для каждой категории разные показатели.
1. children - Идеальным клиентом считается: Взрослая семья (возраст от 31 года) с 5-ю детьми. Худшим клиентом является: Молодая семья (возраст до 30 лет) с (4+) - детьми. Разница между минимальным и максимальным средними значениями равна 64%.
2. family_status - Идеальным клиентом считается: Врозслый человек в разводе (возраст от 31 года). Худшим клиентом является: Молодые люди в разводе (возраст до 30 лет). Разница между минимальным и максимальным средними значениями равна 56,6%.
3. total_income - Идеальным клиентом считается: Врозслый человек c низким доходом (возраст от 31 года). Худшим клиентом является: Молодые люди с доходом выше среднего (возраст до 30 лет). Разница между минимальным и максимальным средними значениями равна 52%.
4. purpose - Идеальным клиентом считается: Врозслый человек c целью покупки недвижимости (возраст от 31 года). Худшим клиентом является: Молодые люди с целью образование (возраст до 30 лет). Разница между минимальным и максимальным средними значениями равна 48,4%.
Исходя из этого анализа большую зависимость имеет Категория - children.

<font color='green'>Интересные данные, очень могут пригодиться заказчику, молодец.</font>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [X]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<font color='blue'>В целом ты отлично поработал. Показал знания большинства принципов, изучаемых в спринте. Ты умеешь изучать и изменять данные, делать выводы и формулировать гипотезы. Умеешь писать собственные функции и применять их в коде.
Учти мои замечания в доработке, и мы доведем проект до идеала. Удачи!</font>

<font color=blue>Молодец, что сделал категоризацию с применением квантилей. Дубликаты убраны, работу принимаю. Удачи в дальнейшем изучении курса! </font>